# Settings

In [1]:
%run Fnc_StartupFabricSempy

StatementMeta(, 255507a8-583a-4692-855a-6b27bc9e0f92, 9, Finished, Available, Finished)

In [2]:
prm_starttime = datetime.now()
NotebookName = "Nbt_Landing_GetColumns"

StatementMeta(, 255507a8-583a-4692-855a-6b27bc9e0f92, 10, Finished, Available, Finished)

In [3]:
Table_Name = 'Landing_Fabric_Columns'

StatementMeta(, 255507a8-583a-4692-855a-6b27bc9e0f92, 11, Finished, Available, Finished)

Deze functie dient om de single quotes in text velden op te vangen en te veranderen voor we problemen krijgen met dit naar de SQL tables te schrijven.

In [4]:
lh_abfs_path = fnc_startUp(Table_Name)

StatementMeta(, 255507a8-583a-4692-855a-6b27bc9e0f92, 12, Finished, Available, Finished)

niet leeg


SynapseWidget(Synapse.DataFrame, 9e780dbd-9e7a-4e2d-933a-00d719cf11cd)

In [5]:
SemanticModels = spark.sql("""select Id, WSID
from Lkh_SemanticLink.Landing_Fabric_Items
where Type='SemanticModel' and DisplayName<>'Report Usage Metrics Model'""")

StatementMeta(, 255507a8-583a-4692-855a-6b27bc9e0f92, 13, Finished, Available, Finished)

In [6]:
for Id, WSID in SemanticModels.toLocalIterator():
    dataset_ID = Id
    try:
        columns = fabric.list_columns(dataset=dataset_ID, workspace=WSID, extended=True)
        columns.drop('Alternate Of Base Column', inplace=True, axis=1)
        columns.drop('Alternate Of Base Table', inplace=True, axis=1)
    except Exception as e:
        print(f"Error fetching semantic model objects for {dataset_ID}: {e}")
        continue
    columnsdf = pd.DataFrame(columns)
    if not columnsdf.empty: # check if the list is not empty to avoid errors
        columnsdf = fnc_PrepareColumns(columnsdf)
        sparkdf = spark.createDataFrame(columnsdf)
        sparkdf = sparkdf.withColumn('WSID', lit(WSID))
        sparkdf = sparkdf.withColumn('SMID', lit(dataset_ID))
        sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")

StatementMeta(, 255507a8-583a-4692-855a-6b27bc9e0f92, 14, Submitted, Running, Running)

In [ ]:
#fill list of workspaces
workspaces = fabric.list_workspaces().sort_values(by='Name', ascending=True)
df = pd.DataFrame(workspaces)

# loop through the workspaces
for label, content in df.items():
    if label == 'Id': #loop through the ID's
        for id in content:
            #lijst vullen met alle items in de werkruimte
            temp_items = fabric.list_items(workspace=id)
            itemdf = pd.DataFrame(temp_items)
            #print(itemdf)
            if not itemdf.empty: # check if the list is not empty to avoid errors
            # filter out semantic models
                df_semantic_models = itemdf[(itemdf['Type'] == 'SemanticModel') & (itemdf['Display Name'] != 'Report Usage Metrics Model')]
                #print(df_semantic_models)
                for _, row in df_semantic_models.iterrows():
                    dataset_name = row['Display Name']
                    dataset_ID = row['Id']
                    #get the measures of each model
                    try:
                        columns = fabric.list_columns(dataset=dataset_ID, workspace=id, extended=True)
                        columns.drop('Alternate Of Base Column', inplace=True, axis=1)
                        columns.drop('Alternate Of Base Table', inplace=True, axis=1)
                    except Exception as e:
                        print(f"Error fetching semantic model objects for {dataset_name}: {e}")
                        continue
                    columnsdf = pd.DataFrame(columns)
                    if not columnsdf.empty: # check if the list is not empty to avoid errors
                        columnsdf = fnc_PrepareColumns(columnsdf)
                        sparkdf = spark.createDataFrame(columnsdf)
                        sparkdf = sparkdf.withColumn('WSID', lit(id))
                        sparkdf = sparkdf.withColumn('SMID', lit(dataset_ID))
                        sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")

                    

In [ ]:
prm_endtime = datetime.utcnow()
fnc_LognotebookRefresh(NotebookName, prm_starttime, prm_endtime)

StatementMeta(, , -1, Waiting, , Waiting)